## Volume Demand Forecast for Multiple Products

### 1. Project purpose

A company has a large portfolio of products distributed to retailers through agencies. There are thousands of unique agency-SKU/product combinations. In order to plan its production and distribution as well as help agencies with their planning, it is important for the company to have an accurate estimate of demand at SKU level for each agency.

Our purpose is to achieve the following using DeepAR algorithm:
- taking advantage of DeepAR's RNN framework. To predict product demand that's not only trained on their own historical data, but also trained on historical data of other variables(e.g., sales price and promotion price) that have impact on product demand.  
- predicting the monthly demand volume(hectoliters) for each Agency-SKU combination (350 Agency-SKU combinations in total) for year 2018 based on data from 2013-2017.
- handling cold start problems, which is to do volume demand forecast for new Agency-SKU combinations that we don't have any data on.  

### 2. Set up Sagemaker instance and load Data

#### 2.1 Sagemaker instance and S3 setup

In [2]:
from __future__ import print_function
%matplotlib inline
from random import shuffle
import random
import datetime
import os
# import boto3
# import s3fs
# import sagemaker
# from sagemaker import get_execution_role
# from sagemaker.amazon.amazon_estimator import get_image_uri
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
# set random seeds for reproducibility
np.random.seed(4216)
random.seed(4216)

In [4]:
#Get the execution role for the notebook instance. This is the IAM role created for the notebook instance that creates an access b/w S3 bucket and this instance
#load our data saved in S3 bucket
df_historical_volume = pd.read_csv("historical_volume.csv")
df_price_sales_promotion = pd.read_csv("price_sales_promotion.csv")


#### 2.2 Loading data
We have two datasets available for analysis. 
- historical_volume.csv : holds volume(hectoliters) sales data at Agency-SKU-month level from Jan 2013 to Dec 2017
- price_sales_promotion.csv : Holds price (dollar/hectoliters), sales(dollar/hectoliters) and promotion(dollar/hectoliter) at Agency-SKU-month level from Jan 2013 to Dec 2017. The relationship between these 3 variables: price = sales + promotion

Data source: https://www.kaggle.com/utathya/future-volume-prediction 

In [5]:
df_historical_volume.sample(3)

,Agency,SKU,YearMonth,Volume
12147,Agency_28,SKU_01,201704,36.828
11652,Agency_26,SKU_03,201604,0.000
67,Agency_50,SKU_01,201301,1144.800


In [7]:
df_price_sales_promotion.sample(5)

,Agency,SKU,YearMonth,Price,Sales,Promotions
8760,Agency_46,SKU_03,201302,1271.634938,1252.793957,18.840981
7330,Agency_21,SKU_03,201404,1332.054913,1100.508520,231.546393
18203,Agency_54,SKU_14,201305,0.000000,0.000000,0.000000
14937,Agency_32,SKU_05,201710,1919.156779,1610.401856,308.754923
11447,Agency_32,SKU_04,201608,2170.270691,1788.125455,382.145236


### 3. Data cleaning
To manipulate data to the correct format required by SageMaker DeepAR 

#### 3.1 Combining two datasets
Combine two datasets by 'Agency', 'SKU', 'YearMonth'.

In [9]:
df1 = df_historical_volume.sort_values(by = ['Agency', 'SKU', 'YearMonth'], inplace = False, ignore_index = True)
df2 = df_price_sales_promotion.sort_values(by = ['Agency', 'SKU', 'YearMonth'], inplace = False, ignore_index = True)

df = pd.merge(df1, df2, how = 'inner', on = ['Agency', 'SKU', 'YearMonth'])

df['time'] = pd.to_datetime(df['YearMonth'], format = '%Y%m')

df.sample(5)

,Agency,SKU,YearMonth,Volume,Price,Sales,Promotions,time
20970,Agency_60,SKU_23,201507,0.0000,0.000000,0.000000,0.000000,2015-07-01
20166,Agency_59,SKU_01,201307,8823.8175,1289.293543,1207.097405,82.196138,2013-07-01
13975,Agency_43,SKU_01,201708,433.9440,1686.687500,1369.531654,317.155846,2017-08-01
19558,Agency_57,SKU_07,201711,134.5200,1593.812940,1356.976004,236.836936,2017-11-01
13828,Agency_42,SKU_04,201505,129.2697,2089.730546,1953.573065,136.157481,2015-05-01


#### 3.2 Label encoding 
Use label encoders for 2 category variables-- Agency and SKU since DeepAR only accepts categories represented by number. There are 58 different agencies and 25 different SKUs. 

In [21]:
from sklearn import preprocessing
#use label encoding to encode 'Agency' and 'SKU' to numbers 
agency_le = preprocessing.LabelEncoder()
df['Agency_le'] = agency_le.fit_transform(df['Agency'])
sku_le = preprocessing.LabelEncoder()
df['SKU_le'] = sku_le.fit_transform(df['SKU'])

# df.tail(10)

# agency_label = agency_le.transform(agency_le.classes_)
# agency_name = agency_le.classes_
# agency_label_encoder = pd.DataFrame(zip(agency_name, agency_label), columns = ['agency_name', 'agency_label']).to_string(index = False)


# sku_label = sku_le.transform(sku_le.classes_)
# sku_name = sku_le.classes_
# sku_label_encoder = pd.DataFrame(zip(sku_name, sku_label), columns = ['sku_name', 'sku_label']).to_string(index = False)

df.sample(5)


/Users/mahsa/anaconda3/envs/playground/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/mahsa/anaconda3/envs/playground/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/mahsa/anaconda3/envs/playground/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/mahsa/anaconda3/envs/playground/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Che

,Agency,SKU,YearMonth,Volume,Price,Sales,Promotions,time,Agency_le,SKU_le
16140,Agency_49,SKU_01,201301,2225.4480,1226.108241,1220.071220,6.037021,2013-01-01,46,0
7238,Agency_21,SKU_21,201603,0.0000,0.000000,0.000000,0.000000,2016-03-01,18,15
137,Agency_01,SKU_03,201406,69.5412,1393.911321,1284.701623,109.209698,2014-06-01,0,2
4241,Agency_12,SKU_05,201606,2992.5645,1520.897131,1239.782100,281.115031,2016-06-01,10,4
7828,Agency_23,SKU_05,201505,828.5910,1486.594316,1272.961305,213.633011,2015-05-01,20,4


#### 3.3 Transforming the data format
We start to transform data from a vertical format to a dictionary format using group_by and merge etc. After grouping the data by 'Agency' and 'SKU', we can see we have 350 time series that we will need to predict simultaneously.

In [39]:
group_by = df.groupby(['Agency_le', 'SKU_le']) #  350 agency-SKU combination time series in total
volume_series = group_by['Volume'].apply(list).reset_index(name = 'volume')
sales_series = group_by['Sales'].apply(list).reset_index(name = 'sales')
promotion_series = group_by['Promotions'].apply(list).reset_index(name = 'promotions')

df_merge = pd.merge(volume_series, sales_series, how= 'inner', on= ['Agency_le','SKU_le']) 
df_merge2 = pd.merge(df_merge, promotion_series, how = 'inner', on = ['Agency_le','SKU_le'])
df_merge2['cat_agency_sku'] = df_merge2[['Agency_le', 'SKU_le']].apply(lambda x: list(x), axis=1)

#check the lengths of each dynamic features.len should be 60.

np.where(df_merge2.volume.apply(lambda x: len(x)) != 60)
np.where(df_merge2.sales.apply(lambda x: len(x)) !=60)
np.where(df_merge2.promotions.apply(lambda x: len(x)) !=60)

df_merge2

,Agency_le,SKU_le,volume,sales,promotions,cat_agency_sku
0,0,0,"[80.676, 98.064, 133.704, 147.312, 175.608, 18...","[1033.432731, 1065.417195, 1101.133633, 1138.2...","[108.067269, 76.082805, 78.212187, 88.404143, ...","[0, 0]"
1,0,1,"[78.408, 99.252, 137.268, 135.864, 167.832, 15...","[969.1862085, 996.9507621, 1061.272227, 1113.6...","[104.9715905, 77.9940829, 67.717594, 87.975954...","[0, 1]"
2,0,2,"[35.955, 45.684, 53.298, 61.8426, 65.142, 94.8...","[1141.894806, 1154.786791, 1183.434397, 1197.8...","[97.839237, 84.947252, 82.374114, 107.075669, ...","[0, 2]"
3,0,3,"[230.2956, 270.4887, 411.4521, 396.5634, 498.0...","[1467.68186, 1500.037283, 1603.69942, 1589.691...","[258.029967, 228.508225, 166.576616, 225.17043...","[0, 3]"
4,0,4,"[15.975, 16.4223, 30.672, 26.5185, 27.4131, 26...","[1172.763381, 1171.803584, 1243.893486, 1242.9...","[87.559436, 86.793582, 64.920555, 110.418633, ...","[0, 4]"
...,...,...,...,...,...,...
345,57,2,"[335.016, 446.9418, 968.247, 580.1868, 689.151...","[1174.171358, 1178.179712, 1213.793473, 1159.6...","[91.544013, 87.646841, 81.781457, 154.011194, ...","[57, 2]"
346,57,3,"[2201.0355, 2269.728, 2594.0205, 2622.648, 280...","[1637.073176, 1631.781117, 1669.963723, 1741.0...","[134.263949, 136.967854, 137.911583, 137.24238...","[57, 3]"
347,57,4,"[2005.3098, 2111.895, 2270.30325, 2408.5185, 2...","[1233.085829, 1228.666919, 1258.891157, 1315.2...","[76.002875, 80.120268, 82.490385, 85.707017, 8...","[57, 4]"
348,57,6,"[35.8425, 56.1975, 70.8, 69.4725, 79.2075, 69....","[1058.74021, 1048.839219, 1095.033887, 1132.93...","[68.537488, 76.783454, 73.649427, 92.264551, 9...","[57, 6]"


In [40]:
# monthly volume, sales and promotions for each agency and sku averaged over 2013-2017
Agency_SKU = df.groupby(['Agency','SKU'])
Agency_SKU_mean = Agency_SKU['Volume'].mean().reset_index(name = 'average_volume')
Agency_SKU_mean.sample(3)

,Agency,SKU,average_volume
314,Agency_55,SKU_05,566.857943
258,Agency_47,SKU_05,0.212940
182,Agency_31,SKU_04,944.800548


#### 3.4 Spliting data to training and test sets
- training set: 48-month data from 2013-01 to 2016-12
- test set: 12-month data from 2017-01 to 2017-12


In [41]:
df_merge2['sales_promotions'] = df_merge2.apply(lambda x: np.vstack([np.array(x['sales']),np.array(x['promotions'])]),axis = 1)
df_merge2['training_volume'] = df_merge2['volume'].apply(lambda x: x[:48])
df_merge2['training_sales'] = df_merge2['sales'].apply(lambda x: x[:48])
df_merge2['training_promotions'] = df_merge2['promotions'].apply(lambda x: x[:48])
df_merge2['training_sales_promotions'] = df_merge2.apply(lambda x: np.vstack([np.array(x['training_sales']),np.array(x['training_promotions'])]),axis = 1)

df_merge2['test_volume'] = df_merge2['volume'].apply(lambda x: x[48:])
df_merge2['test_sales'] = df_merge2['sales'].apply(lambda x: x[48:])
df_merge2['test_promotions'] = df_merge2['promotions'].apply(lambda x: x[48:])
df_merge2['test_sales_promotions'] = df_merge2.apply(lambda x: np.vstack([np.array(x['test_sales']),np.array(x['test_promotions'])]),axis = 1)

# create the dynamic_feat: sales and promotions for batch transform data. 
df_merge2['batch_sales'] = df_merge2.apply(lambda x: x.sales + x.test_sales, axis = 1)
df_merge2['batch_promotions'] = df_merge2.apply(lambda x: x.promotions + x.test_promotions, axis = 1)
df_merge2['batch_sales_promotions'] = df_merge2.apply(lambda x: np.vstack([np.array(x['batch_sales']),np.array(x['batch_promotions'])]),axis = 1)

In [58]:
full_df_clean = pd.concat([df_merge2,Agency_SKU_mean], axis = 1)
full_df_clean.sample(3)

df_merge2["sales_promotions"][0]

df_merge2.sample(5)

len(df_merge2["volume"][0])

print(len(df_merge2["sales_promotions"][0][0]))


df_merge2.sample(5)

60


,Agency_le,SKU_le,volume,sales,promotions,cat_agency_sku,sales_promotions,training_volume,training_sales,training_promotions,training_sales_promotions,test_volume,test_sales,test_promotions,test_sales_promotions,batch_sales,batch_promotions,batch_sales_promotions
307,51,4,"[61.2162, 68.6925, 77.1273, 86.8401, 78.7887, ...","[1315.562498, 1313.954307, 1345.432367, 1382.8...","[0.408201, 0.242516, 0.0, 0.0, 0.0, 0.0, 6.625...","[51, 4]","[[1315.562498, 1313.954307, 1345.432367, 1382....","[61.2162, 68.6925, 77.1273, 86.8401, 78.7887, ...","[1315.562498, 1313.954307, 1345.432367, 1382.8...","[0.408201, 0.242516, 0.0, 0.0, 0.0, 0.0, 6.625...","[[1315.562498, 1313.954307, 1345.432367, 1382....","[88.54875, 97.98375, 105.76845, 129.3336, 101....","[1590.982366, 1498.013334, 1495.149208, 1392.5...","[208.760208, 327.418684, 329.540315, 432.25821...","[[1590.982366, 1498.013334, 1495.149208, 1392....","[1315.562498, 1313.954307, 1345.432367, 1382.8...","[0.408201, 0.242516, 0.0, 0.0, 0.0, 0.0, 6.625...","[[1315.562498, 1313.954307, 1345.432367, 1382...."
73,11,1,"[0.0, 2062.908, 2285.388, 2646.864, 2697.3, 25...","[0.0, 1128.240729, 1144.175252, 1186.580542, 1...","[0.0, 76.790193, 81.790689, 81.515584, 84.8765...","[11, 1]","[[0.0, 1128.240729, 1144.175252, 1186.580542, ...","[0.0, 2062.908, 2285.388, 2646.864, 2697.3, 25...","[0.0, 1128.240729, 1144.175252, 1186.580542, 1...","[0.0, 76.790193, 81.790689, 81.515584, 84.8765...","[[0.0, 1128.240729, 1144.175252, 1186.580542, ...","[4246.344, 5583.06, 5704.56, 6451.272, 6054.69...","[1320.621094, 1317.834057, 1316.536229, 1225.7...","[307.382968, 319.813473, 320.594756, 412.30206...","[[1320.621094, 1317.834057, 1316.536229, 1225....","[0.0, 1128.240729, 1144.175252, 1186.580542, 1...","[0.0, 76.790193, 81.790689, 81.515584, 84.8765...","[[0.0, 1128.240729, 1144.175252, 1186.580542, ..."
138,22,0,"[17.172, 23.976, 26.784, 27.0, 22.032, 10.584,...","[1066.777516, 1069.345439, 1096.779738, 1126.7...","[102.474842, 99.267737, 104.064264, 108.41275,...","[22, 0]","[[1066.777516, 1069.345439, 1096.779738, 1126....","[17.172, 23.976, 26.784, 27.0, 22.032, 10.584,...","[1066.777516, 1069.345439, 1096.779738, 1126.7...","[102.474842, 99.267737, 104.064264, 108.41275,...","[[1066.777516, 1069.345439, 1096.779738, 1126....","[8.532, 5.076, 5.724, 13.176, 4.104, 2.268, 3....","[1464.827531, 1303.945479, 1228.253538, 1202.5...","[311.206488, 340.117021, 415.808962, 441.54149...","[[1464.827531, 1303.945479, 1228.253538, 1202....","[1066.777516, 1069.345439, 1096.779738, 1126.7...","[102.474842, 99.267737, 104.064264, 108.41275,...","[[1066.777516, 1069.345439, 1096.779738, 1126...."
156,24,5,"[64.908, 77.004, 92.664, 106.056, 113.184, 107...","[912.4412166, 902.3444037, 915.351192, 936.528...","[80.407845, 82.8398817, 72.6215364, 92.1922121...","[24, 5]","[[912.4412166, 902.3444037, 915.351192, 936.52...","[64.908, 77.004, 92.664, 106.056, 113.184, 107...","[912.4412166, 902.3444037, 915.351192, 936.528...","[80.407845, 82.8398817, 72.6215364, 92.1922121...","[[912.4412166, 902.3444037, 915.351192, 936.52...","[7.992, 8.424, 8.964, 10.692, 14.256, 10.152, ...","[1319.831081, 1313.798077, 1299.143073, 1298.0...","[88.226351, 108.536058, 73.545933, 69.006314, ...","[[1319.831081, 1313.798077, 1299.143073, 1298....","[912.4412166, 902.3444037, 915.351192, 936.528...","[80.407845, 82.8398817, 72.6215364, 92.1922121...","[[912.4412166, 902.3444037, 915.351192, 936.52..."
68,10,2,"[510.6456, 498.4632, 538.479, 624.2634, 680.69...","[1157.050899, 1156.605128, 1174.192448, 1184.4...","[93.560341, 94.08717, 99.040934, 120.870774, 2...","[10, 2]","[[1157.050899, 1156.605128, 1174.192448, 1184....","[510.6456, 498.4632, 538.479, 624.2634, 680.69...","[1157.050899, 1156.605128, 1174.192448, 1184.4...","[93.560341, 94.08717, 99.040934, 120.870774, 2...","[[1157.050899, 1156.605128, 1174.192448, 1184....","[587.487, 455.19225, 403.9971, 560.9826, 506.5...","[1312.888415, 1396.164439, 1378.896139, 1319.6...","[3

#### 3.5 Transforming training and test sets to dictionary format
Transforming data to dictionary format to be prepared for json line format required by DeepAR

In [422]:
start_training = pd.Timestamp("2013-01-01 00:00:00")


training_data = []
for _,row in df_merge2.iterrows():
    training_dic = {"start": str(start_training),
           "target": row['training_volume'],
           "cat": row['cat_agency_sku'],
           "dynamic_feat": row['training_sales_promotions'] 
           }
    training_data.append(training_dic)

    

test_data = []
for _,row in df_merge2.iterrows():
    test_dic = {"start": str(start_training),
           "target": row['volume'],
           "cat": row['cat_agency_sku'],
           "dynamic_feat": row['sales_promotions'] 
           }
    test_data.append(test_dic)


# create batch transform input: predicting volume demand for each SKU and agency pair for 201801 - 201812 given dynamic_feat--'sales' and 'promotions' remain the same as of 2017
batch_transform_input = []
for _,row in df_merge2.iterrows():
    batch_dic = {"start": str(start_training),
           "target": row['volume'],
           "cat": row['cat_agency_sku'],
           "dynamic_feat": row['batch_sales_promotions'] 
           }
    batch_transform_input.append(batch_dic)

    
    
#dataset for calculating predicted volume for each series in the test set
data_for_calculating_predicted = []
for _,row in df_merge2.iterrows():
    calculating_predicted_dic = {"start": str(start_training),
           "target": row['training_volume'],
           "cat": row['cat_agency_sku'],
           "dynamic_feat": row['sales_promotions'] 
           }
    data_for_calculating_predicted.append(calculating_predicted_dic)

#### 3.6 Transforming dictionary to json lines

In [423]:
import json 

# correct np narray
class NumpyEncoder(json.JSONEncoder):
    """ Special json encoder for numpy types """
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

# convert to json lines: each line is a json object
def write_dicts_to_file(path, data):
    with open(path, 'wb') as fp:
        for d in data:
            fp.write(json.dumps(d, cls=NumpyEncoder).encode("utf-8"))
            fp.write("\n".encode('utf-8'))

write_dicts_to_file("train.json", training_data)
write_dicts_to_file("test.json", test_data)
write_dicts_to_file("batch_transform_input.json", batch_transform_input)
write_dicts_to_file("data_for_calculating_predicted.json", data_for_calculating_predicted)

#### 3.7 Copying the local json lines to S3 where DeepAR can access

In [424]:
import logging
import boto3
from botocore.exceptions import ClientError


def upload_file(file_name, bucket, object_name=None):
    """Upload a file to an S3 bucket

    :param file_name: File to upload
    :param bucket: Bucket to upload to
    :param object_name: S3 object name. If not specified then file_name is used
    :return: True if file was uploaded, else False
    """

    # If S3 object_name was not specified, use file_name
    if object_name is None:
        object_name = file_name

    # Upload the file
    s3_client = boto3.client('s3')
    try:
        response = s3_client.upload_file(file_name, bucket, object_name)
    except ClientError as e:
        logging.error(e)
        return False
    return True

In [425]:
BUCKET_NAME = 'sagemaker-us-east-2-939910653043'
upload_file("train.json", BUCKET_NAME, object_name= "product-demand-forecast-deepar/data/train.json")
upload_file("test.json", BUCKET_NAME, object_name= "product-demand-forecast-deepar/data/test.json")
upload_file("batch_transform_input.json", BUCKET_NAME, object_name= "product-demand-forecast-deepar/batch_transform_input.json")
upload_file("data_for_calculating_predicted.json", BUCKET_NAME, object_name= "product-demand-forecast-deepar/data_for_calculating_predicted.json")

True

In [541]:
# have a look to what was just written to S3. Now data has been successfully customized to json line which is required by DeepAR
s3filesystem = s3fs.S3FileSystem()
with s3filesystem.open(BUCKET_NAME + "/product-demand-forecast-deepar/data/test.json", 'rb') as fp:
    print(fp.readline().decode("utf-8")[:500] + "...")

{"start": "2013-01-01 00:00:00", "target": [80.676, 98.064, 133.704, 147.312, 175.608, 180.792, 149.796, 161.136, 147.744, 149.364, 95.36399999999999, 104.544, 100.54799999999999, 123.552, 125.17200000000001, 70.524, 21.924, 66.744, 89.1, 107.352, 100.87200000000001, 113.292, 68.148, 75.492, 41.58, 58.32, 87.37200000000001, 81.0, 91.90799999999999, 86.616, 135.0, 97.632, 103.464, 92.016, 54.864, 40.608000000000004, 55.836000000000006, 52.812, 88.992, 66.63600000000001, 49.14, 4.32, 54.4319999999...
